In [1]:
# Enhanced Language File Processor with Configurable Filtering
# Supports Excel (.xlsx, .xls) and XLIFF (.xliff, .xlf, .xml) files

import pandas as pd
import xml.etree.ElementTree as ET
import re
import os
import time
from pathlib import Path
from typing import Set, List, Tuple
import html

def remove_html_tags(text: str) -> str:
    """Remove HTML tags and decode HTML entities, with space insertion for br/p tags"""
    if not text:
        return text
    
    # First, replace br and p tags with spaces to prevent word concatenation
    # Handle both self-closing and regular br tags
    text = re.sub(r'&lt;/?br\s*/?&gt;', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'&lt;/?p\s*/?&gt;', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'&lt;p\s+[^&]*&gt;', ' ', text, flags=re.IGNORECASE)  # p with attributes
    text = re.sub(r'&lt;/p&gt;', ' ', text, flags=re.IGNORECASE)
    
    # Remove other HTML tags (without space insertion)
    text = re.sub(r'&lt;[^&]*&gt;', '', text)
    
    # Decode HTML entities
    text = html.unescape(text)
    
    # Clean up multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def matches_time_pattern(token: str) -> bool:
    """Check if token matches time pattern like 3PM, 10AM, 5PA, 12AL"""
    return bool(re.match(r'^\d+(PM|AM|PA|AL)$', token, re.IGNORECASE))

def matches_digit_word_pattern(token: str) -> bool:
    """Check if token matches digit-word pattern like 123-neutral"""
    return bool(re.match(r'^\d+-\w+$', token))

def process_english_contractions(text: str) -> str:
    """Process English contractions while preserving case"""
    if not text:
        return text
    
    # Comprehensive English contractions mapping
    contractions = {
        "ain't": "am not", "aren't": "are not", "can't": "cannot", "could've": "could have",
        "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not",
        "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
        "he'll": "he will", "he's": "he is", "i'd": "i would", "i'll": "i will", "i'm": "i am",
        "i've": "i have", "isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is",
        "let's": "let us", "mustn't": "must not", "shan't": "shall not", "she'd": "she would",
        "she'll": "she will", "she's": "she is", "shouldn't": "should not", "that's": "that is",
        "there's": "there is", "they'd": "they would", "they'll": "they will", "they're": "they are",
        "they've": "they have", "we'd": "we would", "we're": "we are", "we've": "we have",
        "weren't": "were not", "what's": "what is", "where's": "where is", "who's": "who is",
        "won't": "will not", "wouldn't": "would not", "you'd": "you would", "you'll": "you will",
        "you're": "you are", "you've": "you have", "'cause": "because", "how's": "how is",
        "when's": "when is", "why's": "why is", "y'all": "you all", "would've": "would have",
        "should've": "should have", "might've": "might have", "must've": "must have"
    }
    
    def replace_contraction(match):
        contraction = match.group(0)
        lower_contraction = contraction.lower()
        
        if lower_contraction in contractions:
            replacement = contractions[lower_contraction]
            
            # Preserve case: if original was capitalized, capitalize the replacement
            if contraction[0].isupper():
                replacement = replacement.capitalize()
            
            return replacement
        return contraction
    
    # Use word boundaries to match contractions
    pattern = r"\b(?:" + "|".join(re.escape(cont) for cont in contractions.keys()) + r")\b"
    result = re.sub(pattern, replace_contraction, text, flags=re.IGNORECASE)
    
    return result

def process_portuguese_contractions(text: str) -> str:
    """Process Portuguese contractions and apostrophe patterns"""
    if not text:
        return text
    
    # Handle apostrophe contractions like d'Água -> de Água
    text = re.sub(r"\bd'([A-ZÁÉÍÓÚÂÊÔÀÇ])", r"de \1", text)
    text = re.sub(r"\bl'([A-ZÁÉÍÓÚÂÊÔÀÇ])", r"le \1", text)
    
    # Handle hyphenated pronouns like amá-lo -> amar lo
    text = re.sub(r"([aeiouáéíóúâêôàç])-([lm][eoasá]s?)\b", r"\1r \2", text)
    
    return text

def has_wip_markers(text: str) -> bool:
    """Check if text contains WIP/translation markers"""
    if not text:
        return False
    
    # Pattern to match markers like {WIP}, [NOTRAD], [no trad], {no_trad}, etc.
    pattern = r'[\[\{].*(wip|notrad|no trad|no_trad).*[\]\}]'
    return bool(re.search(pattern, text, re.IGNORECASE))

def tokenize_text(text: str, language: str = "default") -> Set[str]:
    """
    Enhanced tokenize function with language-specific processing and comprehensive filtering
    
    Args:
        text: Input text to tokenize
        language: Language for processing ("english", "portuguese", or "default")
    
    Returns:
        Set of filtered tokens
    """
    if not text or not isinstance(text, str):
        return set()
    
    # Step 1: Remove HTML tags and decode entities
    text = remove_html_tags(text)
    
    # Step 2: Remove URLs and email addresses
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    
    # Step 3: Language-specific contraction processing
    if language.lower() == "english":
        text = process_english_contractions(text)
    elif language.lower() == "portuguese":
        text = process_portuguese_contractions(text)
    # For "default" or other languages, skip contraction processing
    
   # Step 4: Enhanced punctuation (including º character)
    basic_punct = '.,;:¡!?""''()[]{}«»„"‚-+=*/@#$%^&|\\<>~`º'
    basic_punct += "“”‘’"  # Adding curly and single quotes
    unicode_dashes = '\u2014\u2013'  # em-dash and en-dash
    punctuation = basic_punct + unicode_dashes
    
    # Step 5: Tokenize by whitespace and punctuation, preserving internal hyphens and apostrophes
    tokens = re.findall(r"[^\s" + re.escape(punctuation) + r"]+(?:[-'][^\s" + re.escape(punctuation) + r"]+)*", text)

    
    # Step 6: Clean and filter tokens
    filtered_tokens = set()
    for token in tokens:
        # Remove leading/trailing apostrophes and hyphens
        cleaned_token = token.strip("'-")
        
        # Skip if empty after cleaning
        if not cleaned_token:
            continue
        
        # Skip short tokens (< 3 characters)
        if len(cleaned_token) < 3:
            continue
        
        # Skip tokens that are chains of the same character
        if len(set(cleaned_token.lower())) == 1:
            continue
        
        # Skip tokens that are only digits
        if cleaned_token.isdigit():
            continue
        
        # Skip time patterns (e.g., "3PM", "10AM", "5PA", "12AL")
        if matches_time_pattern(cleaned_token):
            continue
        
        # Skip digit-word patterns (e.g., "123-neutral")
        if matches_digit_word_pattern(cleaned_token):
            continue
        
        filtered_tokens.add(cleaned_token)
    
    return filtered_tokens

def detect_file_type(file_path: str) -> str:
    """Detect if file is Excel or XLIFF based on extension"""
    file_path_lower = file_path.lower()
    if file_path_lower.endswith(('.xlsx', '.xls')):
        return 'excel'
    elif file_path_lower.endswith(('.xliff', '.xlf', '.xml')):
        return 'xliff'
    else:
        raise ValueError(f"Unsupported file type for: {file_path}")

def process_excel_file(file_path: str, language_code: str, ignore_identical_translation: bool, 
                      tokenize_language: str, skip_square_brackets: bool, skip_all_caps: bool, 
                      skip_wip_markers: bool) -> Tuple[Set[str], int, int]:
    """Process Excel file and extract tokens with configurable filtering"""
    
    # Try to find the sheet with actual data for the language
    xl_file = pd.ExcelFile(file_path)
    df = None
    sheet_used = None
    
    for sheet_name in xl_file.sheet_names:
        temp_df = pd.read_excel(file_path, sheet_name=sheet_name)
        if language_code in temp_df.columns:
            non_null_count = temp_df[language_code].notna().sum()
            if non_null_count > 0:
                df = temp_df
                sheet_used = sheet_name
                print(f"Using sheet '{sheet_name}' with {non_null_count} {language_code} values")
                break
    
    if df is None:
        # Fallback to default sheet
        df = pd.read_excel(file_path)
        sheet_used = "default"
    
    print(f"Excel columns: {list(df.columns)}")
    print(f"Sheet used: {sheet_used}")
    
    if language_code not in df.columns:
        raise ValueError(f"Language code '{language_code}' not found in Excel columns: {list(df.columns)}")
    
    print(f"Total Excel rows to process: {len(df)}")
    
    # Initialize tracking
    tokens = set()
    processed_count = 0
    skipped_count = 0
    skip_reasons = {"identical": 0, "square_brackets": 0, "all_caps": 0, "wip_markers": 0, "empty_target": 0}
    
    for index, row in df.iterrows():
        source_text = str(row.iloc[1]) if len(row) > 1 else ""  # Assume source is second column
        
        # Check if target is NaN or empty BEFORE converting to string
        target_value = row[language_code]
        if pd.isna(target_value):
            skipped_count += 1
            skip_reasons["empty_target"] += 1
            continue
            
        target_text = str(target_value)
        
        # Skip if target is empty string after conversion
        if target_text.strip() == '':
            skipped_count += 1
            skip_reasons["empty_target"] += 1
            continue
        
        # Apply filters based on configuration
        should_skip = False
        skip_reason = None
        
        # Filter 1: Identical translation
        if ignore_identical_translation and source_text == target_text:
            should_skip = True
            skip_reason = "identical"
        
        # Filter 2: Square brackets in source
        elif skip_square_brackets and re.search(r'\[.+\]', source_text):
            should_skip = True
            skip_reason = "square_brackets"
        
        # Filter 3: All caps target
        elif skip_all_caps and target_text.isupper() and len(target_text) > 2:
            should_skip = True
            skip_reason = "all_caps"
        
        # Filter 4: WIP markers
        elif skip_wip_markers and has_wip_markers(target_text):
            should_skip = True
            skip_reason = "wip_markers"
        
        if should_skip:
            skipped_count += 1
            skip_reasons[skip_reason] += 1
            continue
        
        # Process the target text
        processed_count += 1
        text_tokens = tokenize_text(target_text, tokenize_language)
        tokens.update(text_tokens)
    
    # Print skip statistics
    print(f"Skip reasons breakdown:")
    for reason, count in skip_reasons.items():
        if count > 0:
            print(f"  - {reason}: {count}")
    
    return tokens, processed_count, skipped_count

def process_xliff_file(file_path: str, language_code: str, ignore_identical_translation: bool,
                      tokenize_language: str, skip_square_brackets: bool, skip_all_caps: bool,
                      skip_wip_markers: bool) -> Tuple[Set[str], int, int]:
    """Process XLIFF file and extract tokens with configurable filtering"""
    
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Find the namespace
    namespace = ''
    if root.tag.startswith('{'):
        namespace = root.tag.split('}')[0] + '}'
    
    # Find file element and check language attributes
    file_elem = root.find(f'.//{namespace}file')
    if file_elem is None:
        raise ValueError("No file element found in XLIFF")
    
    source_lang = file_elem.get('source-language', '')
    target_lang = file_elem.get('target-language', '')
    
    print(f"XLIFF source language: {source_lang}")
    print(f"XLIFF target language: {target_lang}")
    
    # Determine if we should extract from source or target elements
    use_source = (language_code == source_lang)
    use_target = (language_code == target_lang)
    
    if not (use_source or use_target):
        raise ValueError(f"Language code '{language_code}' not found in XLIFF languages: {source_lang}, {target_lang}")
    
    # Find all trans-unit elements
    trans_units = root.findall(f'.//{namespace}trans-unit')
    print(f"Total XLIFF segments to process: {len(trans_units)}")
    
    # Initialize tracking
    tokens = set()
    processed_count = 0
    skipped_count = 0
    skip_reasons = {"identical": 0, "square_brackets": 0, "all_caps": 0, "wip_markers": 0}
    
    for trans_unit in trans_units:
        source_elem = trans_unit.find(f'{namespace}source')
        target_elem = trans_unit.find(f'{namespace}target')
        
        source_text = source_elem.text if source_elem is not None and source_elem.text else ""
        target_text = target_elem.text if target_elem is not None and target_elem.text else ""
        
        # Determine which text to process
        text_to_process = source_text if use_source else target_text
        
        # Skip if text is empty
        if not text_to_process:
            skipped_count += 1
            continue
        
        # Apply filters based on configuration
        should_skip = False
        skip_reason = None
        
        # Filter 1: Identical translation (only relevant for target)
        if ignore_identical_translation and use_target and source_text == target_text:
            should_skip = True
            skip_reason = "identical"
        
        # Filter 2: Square brackets in source
        elif skip_square_brackets and re.search(r'\[.+\]', source_text):
            should_skip = True
            skip_reason = "square_brackets"
        
        # Filter 3: All caps target (only relevant for target)
        elif skip_all_caps and use_target and target_text.isupper() and len(target_text) > 2:
            should_skip = True
            skip_reason = "all_caps"
        
        # Filter 4: WIP markers
        elif skip_wip_markers and has_wip_markers(text_to_process):
            should_skip = True
            skip_reason = "wip_markers"
        
        if should_skip:
            skipped_count += 1
            skip_reasons[skip_reason] += 1
            continue
        
        # Process the text
        processed_count += 1
        text_tokens = tokenize_text(text_to_process, tokenize_language)
        tokens.update(text_tokens)
    
    # Print skip statistics
    print(f"Skip reasons breakdown:")
    for reason, count in skip_reasons.items():
        if count > 0:
            print(f"  - {reason}: {count}")
    
    return tokens, processed_count, skipped_count

def export_tokens_to_txt(tokens: Set[str], output_path: str):
    """Export tokens to a text file, one per line, sorted alphabetically"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for token in sorted(tokens):
            f.write(token + '\n')
    print(f"Exported {len(tokens)} unique tokens to: {output_path}")

# Create sample files for demonstration
def create_sample_xliff():
    """Create a sample XLIFF file for testing"""
    sample_xliff_content = """<?xml version="1.0" encoding="UTF-8"?>
<xliff version="1.2" xmlns="urn:oasis:names:tc:xliff:document:1.2">
    <file datatype="plaintext" original="sample" source-language="fr-fr" target-language="es-es">
        <body>
            <trans-unit id="sample.1">
                <source>Votre alignement est probablement au sommet, vos ennemis n'existent plus à l'Apogée.</source>
                <target>Tu alineamiento está probablemente en la cumbre, tus enemigos no existen en el Apogeo.</target>
            </trans-unit>
            <trans-unit id="sample.2">
                <source>Test avec des crochets [DEBUG] dans le source</source>
                <target>Prueba con corchetes en el origen</target>
            </trans-unit>
        </body>
    </file>
</xliff>"""
    
    with open("sample.xliff", "w", encoding="utf-8") as f:
        f.write(sample_xliff_content)
    print("Sample XLIFF file created!")



In [43]:
def process_file(file_path: str, language_code: str, output_path: str = None, 
                ignore_identical_translation: bool = True, tokenize_language: str = "default",
                skip_square_brackets: bool = True, skip_all_caps: bool = True, 
                skip_wip_markers: bool = True):
    """
    Main function to process a file and extract tokens for a given language code
    
    Args:
        file_path: Path to the Excel or XLIFF file
        language_code: Language code (e.g., "es-es")
        output_path: Optional output path for the txt file
        ignore_identical_translation: If True (default), skip entries where target equals source
        tokenize_language: Language for tokenization processing ("english", "portuguese", or "default")
        skip_square_brackets: If True (default), skip entries with square brackets in source
        skip_all_caps: If True (default), skip entries with all-caps target text
        skip_wip_markers: If True (default), skip entries with WIP/NOTRAD markers
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    # Start timing
    start_time = time.time()
    print(f"Processing started at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")
    
    # Print filter configuration
    print(f"\nFilter configuration:")
    print(f"  - Skip identical translations: {ignore_identical_translation}")
    print(f"  - Skip square brackets: {skip_square_brackets}")
    print(f"  - Skip all caps: {skip_all_caps}")
    print(f"  - Skip WIP markers: {skip_wip_markers}")
    print(f"  - Tokenization language: {tokenize_language}")
    
    # Detect file type
    file_type = detect_file_type(file_path)
    print(f"Detected file type: {file_type}")
    
    # Process file based on type
    if file_type == 'excel':
        tokens, processed_count, skipped_count = process_excel_file(
            file_path, language_code, ignore_identical_translation, tokenize_language,
            skip_square_brackets, skip_all_caps, skip_wip_markers)
        entry_type = "rows"
    elif file_type == 'xliff':
        tokens, processed_count, skipped_count = process_xliff_file(
            file_path, language_code, ignore_identical_translation, tokenize_language,
            skip_square_brackets, skip_all_caps, skip_wip_markers)
        entry_type = "segments"
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    
    # Calculate timing
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"\nProcessing completed at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))}")
    print(f"Total processing time: {duration:.2f} seconds ({duration/60:.2f} minutes)")
    print(f"Processing statistics:")
    print(f"  - Processed {entry_type}: {processed_count:,}")
    print(f"  - Skipped {entry_type}: {skipped_count:,}")
    print(f"  - Total {entry_type}: {processed_count + skipped_count:,}")
    if duration > 0:
        print(f"  - Processing rate: {(processed_count + skipped_count)/duration:.1f} {entry_type}/second")
    print(f"  - Found {len(tokens):,} unique tokens for language: {language_code}")
    
    # Generate output path if not provided
    if output_path is None:
        base_name = Path(file_path).stem
        output_path = f"{base_name}_{language_code}_tokens.txt"
    
    # Export tokens
    export_tokens_to_txt(tokens, output_path)
    
    return tokens

In [21]:
# Demonstration with Excel file and new configurable filtering
print("\n" + "="*50)
print("Testing with Excel file and configurable filters:")

# Create sample Excel data with various filter test cases
sample_data = {
    'key': ['normal_text', 'wip_test', 'square_brackets', 'all_caps', 'identical', 'english_contractions'],
    'en-us': ["I can't believe it's working!", "This is {WIP} content", "Normal English text", "SHOUTING TEXT", "Same content", "We don't know what's happening."],
    'es-es': ['¡Hola mundo!', 'Este es contenido [NOTRAD]', 'Texto normal en español', 'TEXTO EN MAYÚSCULAS', 'Same content', 'No sabemos qué está pasando'],
    'pt-br': ["Texto normal em português", "Conteúdo {no_trad} aqui", "Como vai você?", "TEXTO EM MAIÚSCULAS", "Conteúdo idêntico", "Encontrei-me com d'Artagnan"],
    'fr-fr': ['Bonjour le monde!', 'Contenu {WIP} ici', '[Debug] texte normal', 'TEXTE EN MAJUSCULES', 'Conteúdo idêntico', 'Texte français normal']
}

df = pd.DataFrame(sample_data)
df.to_excel("sample_filter_test.xlsx", index=False)
print("Sample Excel file with filter test cases created!")
print(f"Excel columns: {list(df.columns)}")
print("Sample data:")
print(df.to_string(index=False))

print(f"\n" + "="*60)
print("TESTING WIP MARKERS FILTER")
print("="*60)

# Test has_wip_markers function
wip_test_cases = [
    "Normal text without markers",
    "Text with {WIP} marker",
    "Content [NOTRAD] here", 
    "Some {no trad} content",
    "Text with [no_trad] marker",
    "Mixed content {WIP} and more text",
    "Text [WIP] in brackets"
]

print("Testing WIP marker detection:")
for text in wip_test_cases:
    has_wip = has_wip_markers(text)
    print(f"'{text}' -> Has WIP markers: {has_wip}")

print(f"\n" + "="*60)
print("TESTING CONFIGURABLE FILTERS")
print("="*60)

# Test with all filters enabled (default)
print(f"\n1. Processing with ALL filters enabled:")
try:
    tokens_all_filters = process_file("sample_filter_test.xlsx", "es-es", "tokens_all_filters.txt", 
                                    ignore_identical_translation=True,
                                    skip_square_brackets=True,
                                    skip_all_caps=True,
                                    skip_wip_markers=True)
    print(f"Tokens with all filters: {sorted(tokens_all_filters)}")
except Exception as e:
    print(f"Error: {e}")

# Test with no filters (process everything)
print(f"\n2. Processing with NO filters:")
try:
    tokens_no_filters = process_file("sample_filter_test.xlsx", "pt-br", "tokens_no_filters.txt",
                                   ignore_identical_translation=False,
                                   skip_square_brackets=False,
                                   skip_all_caps=False,
                                   skip_wip_markers=False)
    print(f"Tokens with no filters: {sorted(tokens_no_filters)}")
except Exception as e:
    print(f"Error: {e}")

# Test with only WIP filter
print(f"\n3. Processing with ONLY WIP filter:")
try:
    tokens_wip_only = process_file("sample_filter_test.xlsx", "pt-br", "tokens_wip_only.txt",
                                 ignore_identical_translation=False,
                                 skip_square_brackets=False,
                                 skip_all_caps=False,
                                 skip_wip_markers=True)
    print(f"Tokens with WIP filter only: {sorted(tokens_wip_only)}")
except Exception as e:
    print(f"Error: {e}")

# Show differences
if 'tokens_all_filters' in locals() and 'tokens_no_filters' in locals():
    filtered_out = tokens_no_filters - tokens_all_filters
    print(f"\nTokens filtered out by all filters: {sorted(filtered_out)}")

if 'tokens_wip_only' in locals() and 'tokens_no_filters' in locals():
    wip_filtered = tokens_no_filters - tokens_wip_only
    print(f"Tokens filtered out by WIP filter only: {sorted(wip_filtered)}")

# Test English processing with configurable filters
print(f"\n" + "="*60)
print("TESTING ENGLISH WITH CONFIGURABLE FILTERS")
print("="*60)
try:
    print(f"\nProcessing Excel for en-us with English language processing and selective filters:")
    tokens_excel_en = process_file("sample_filter_test.xlsx", "en-us", "excel_english_selective.txt", 
                                 ignore_identical_translation=True,
                                 tokenize_language="english",
                                 skip_square_brackets=False,  # Allow square brackets
                                 skip_all_caps=True,          # Skip all caps
                                 skip_wip_markers=True)       # Skip WIP markers
    print(f"Extracted English tokens: {sorted(tokens_excel_en)}")
    
except Exception as e:
    print(f"Error: {e}")

# Clean up all files
print("\n" + "="*50)
print("Cleaning up files...")
files_to_remove = [
    "sample.xliff", "sample_filter_test.xlsx", 
    "spanish_tokens.txt", "french_tokens.txt",
    "tokens_all_filters.txt", "tokens_no_filters.txt", "tokens_wip_only.txt",
    "excel_english_selective.txt"
]

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed: {file}")

print("\nAll demonstrations completed successfully!")
print("\nSUMMARY:")
print("- The script can handle both Excel (.xlsx, .xls) and XLIFF (.xliff, .xlf, .xml) files")
print("- NEW: Configurable filtering with individual control over each filter")
print("- NEW: WIP marker detection for {WIP}, [NOTRAD], [no trad], [no_trad] patterns")
print("- NEW: Detailed skip statistics showing why entries were filtered")
print("- Language-specific contraction processing for English and Portuguese")
print("- Comprehensive timing and progress reporting")
print("\nFilter options:")
print("- ignore_identical_translation: Skip entries where target equals source")
print("- skip_square_brackets: Skip entries with square brackets in source")
print("- skip_all_caps: Skip entries with all-caps target text") 
print("- skip_wip_markers: Skip entries with WIP/translation markers")
print("\nUsage examples:")
print("# All filters enabled (default)")
print("process_file('file.xlsx', 'es-es')")
print("")
print("# Selective filtering")
print("process_file('file.xlsx', 'es-es', skip_wip_markers=True, skip_all_caps=False)")
print("")
print("# No filtering")
print("process_file('file.xlsx', 'es-es', ignore_identical_translation=False,")
print("             skip_square_brackets=False, skip_all_caps=False, skip_wip_markers=False)")


Testing with Excel file and configurable filters:
Sample Excel file with filter test cases created!
Excel columns: ['key', 'en-us', 'es-es', 'pt-br', 'fr-fr']
Sample data:
                 key                           en-us                       es-es                       pt-br                 fr-fr
         normal_text   I can't believe it's working!                ¡Hola mundo!   Texto normal em português     Bonjour le monde!
            wip_test           This is {WIP} content  Este es contenido [NOTRAD]     Conteúdo {no_trad} aqui     Contenu {WIP} ici
     square_brackets             Normal English text     Texto normal en español              Como vai você?  [Debug] texte normal
            all_caps                   SHOUTING TEXT         TEXTO EN MAYÚSCULAS         TEXTO EM MAIÚSCULAS   TEXTE EN MAJUSCULES
           identical                    Same content                Same content           Conteúdo idêntico     Conteúdo idêntico
english_contractions We don't know what's

# Get word list from language file (TB excel or TM/project XLIFF)

In [44]:
LANGFILE_PATH = r"C:\Users\Nelso\Downloads\2025-06-13_Retro_TB_as at 6 May 2024.xlsx" # Excel file path (terminology base)
LANGFILE_PATH = r"TB_ANK_202507/2025.07.09_TOUCH.xlsx"  # Path to the sample XLIFF file
LANG_CODE = "pt-br"
#EXPORT_PATH = r"C:\Users\Nelso\Downloads\spanishTB_tokens.txt"
EXPORT_FOLDER = "output"

tokenization_lang = "default"  if LANG_CODE[:2] not in ["en", "pt"] else ("english" if LANG_CODE[:2] == "en" else "portuguese")

if not os.path.exists(EXPORT_FOLDER):
    os.makedirs(EXPORT_FOLDER)

time_stamp = time.strftime("%Y%m%d_%H%M%S")
EXPORT_PATH = os.path.join(EXPORT_FOLDER, f"{LANG_CODE}_TOUCH_tokens_{time_stamp}.txt")
# Process the sample file for Spanish (es-es)
try:
    tokens = process_file(LANGFILE_PATH, LANG_CODE, EXPORT_PATH, ignore_identical_translation=False,
                          tokenize_language=tokenization_lang, skip_square_brackets=False, skip_all_caps=False, skip_wip_markers=True)
    #print(f"\nExtracted tokens: {sorted(tokens)}")
    
    # Show the content of the output file
    #with open("spanish_tokens.txt", "r", encoding="utf-8") as f:
     #   content = f.read()
    #print(f"\nContent of spanish_tokens.txt:\n{content}")
    
except Exception as e:
    print(f"Error: {e}")

Processing started at: 2025-09-14 19:18:21

Filter configuration:
  - Skip identical translations: False
  - Skip square brackets: False
  - Skip all caps: False
  - Skip WIP markers: True
  - Tokenization language: portuguese
Detected file type: excel
Excel columns: ['key', 'fr-fr', 'en-gb', 'es-es', 'de-de', 'it-it', 'pt-br']
Total Excel rows to process: 32609
Skip reasons breakdown:
  - empty_target: 32609

Processing completed at: 2025-09-14 19:18:27
Total processing time: 5.82 seconds (0.10 minutes)
Processing statistics:
  - Processed rows: 0
  - Skipped rows: 32,609
  - Total rows: 32,609
  - Processing rate: 5603.6 rows/second
  - Found 0 unique tokens for language: pt-br
Exported 0 unique tokens to: output\pt-br_TOUCH_tokens_20250914_191821.txt


## Batch processing - Get word list from all suppported files from folder
Languages to process : EN, PT, ES

In [46]:
import glob
import os
import time
from pathlib import Path

# Configuration
FOLDER_PATH = "TB_ANK_202507"
TARGET_LANG_CODES = ["pt-br", "pt-BR", "en-us", "en-gb", "en-GB", "es-es", "es-ES", "en-US"]  # Add other languages as needed
EXPORT_FOLDER = "output"

def extract_game_name(filename: str) -> str:
    """Extract game name from filename after first underscore until next underscore or dot"""
    # Remove file extension first
    name_without_ext = Path(filename).stem
    
    # Split by underscore and get the second part (index 1)
    parts = name_without_ext.split('_')
    if len(parts) >= 2:
        # Get second part and clean it up (remove any spaces or special chars that might cause issues)
        game_name = parts[1].replace(' ', '_').replace('-', '_')
        return game_name
    return "unknown"

def normalize_language_code(lang_code: str) -> str:
    """Normalize language codes to standard format"""
    # Convert to lowercase and replace underscores with hyphens
    normalized = lang_code.lower().replace('_', '-')
    return normalized

def get_tokenization_language(lang_code: str) -> str:
    """Determine tokenization language based on language code"""
    lang_prefix = lang_code[:2].lower()
    if lang_prefix == "en":
        return "english"
    elif lang_prefix == "pt":
        return "portuguese"
    else:
        return "default"

def process_all_xlsx_files():
    """Process all xlsx files in the folder for all target language codes"""
    
    # Create output folder if it doesn't exist
    if not os.path.exists(EXPORT_FOLDER):
        os.makedirs(EXPORT_FOLDER)
    
    # Get all xlsx files in the folder
    xlsx_files = glob.glob(os.path.join(FOLDER_PATH, "*.xlsx"))
    
    if not xlsx_files:
        print(f"No xlsx files found in folder: {FOLDER_PATH}")
        return
    
    print(f"Found {len(xlsx_files)} xlsx files to process")
    print(f"Target language codes: {TARGET_LANG_CODES}")
    print("="*70)
    
    # Track overall statistics
    total_processed = 0
    total_errors = 0
    
    # Process each file
    for xlsx_file in xlsx_files:
        filename = os.path.basename(xlsx_file)
        game_name = extract_game_name(filename)
        
        print(f"\n📁 Processing file: {filename}")
        print(f"🎮 Extracted game name: {game_name}")
        
        # Try each target language code
        for lang_code in TARGET_LANG_CODES:
            normalized_lang = normalize_language_code(lang_code)
            tokenization_lang = get_tokenization_language(normalized_lang)
            
            print(f"\n  🌐 Trying language code: {lang_code} (normalized: {normalized_lang})")
            
            try:
                # Generate timestamped export path with game name
                time_stamp = time.strftime("%Y%m%d_%H%M%S")
                export_filename = f"{normalized_lang}_{game_name}_tokens_{time_stamp}.txt"
                export_path = os.path.join(EXPORT_FOLDER, export_filename)
                
                # Skip if file already exists ignoring timestamp
                export_filename_no_timestamp = f"{normalized_lang}_{game_name}_tokens"
                regexp_pattern = re.compile(rf"{re.escape(export_filename_no_timestamp)}_\d{{8}}_\d{{6}}\.txt")
                existing_files = [f for f in os.listdir(EXPORT_FOLDER) if regexp_pattern.match(f)]
                if existing_files:
                    print(f"  ⏭️  Output file already exists: {export_filename} - skipping")
                    continue
                # Process the file
                tokens = process_file(
                    xlsx_file, 
                    lang_code,  # Use original language code for column matching
                    export_path,
                    ignore_identical_translation=False,
                    tokenize_language=tokenization_lang,
                    skip_square_brackets=False,
                    skip_all_caps=False,
                    skip_wip_markers=True
                )
                
                print(f"  ✅ Successfully processed {lang_code}: {len(tokens)} tokens exported to {export_filename}")
                total_processed += 1
                
            except ValueError as e:
                if "not found in Excel columns" in str(e):
                    print(f"  ⏭️  Language code {lang_code} not found in file columns - skipping")
                else:
                    print(f"  ❌ Error processing {lang_code}: {e}")
                    total_errors += 1
            except Exception as e:
                print(f"  ❌ Unexpected error processing {lang_code}: {e}")
                total_errors += 1
    
    # Print final summary
    print("\n" + "="*70)
    print("📊 PROCESSING SUMMARY")
    print("="*70)
    print(f"Total files found: {len(xlsx_files)}")
    print(f"Total language processing attempts: {len(xlsx_files) * len(TARGET_LANG_CODES)}")
    print(f"Successful exports: {total_processed}")
    print(f"Errors encountered: {total_errors}")
    print(f"Skipped (language not found): {len(xlsx_files) * len(TARGET_LANG_CODES) - total_processed - total_errors}")
    
    if total_processed > 0:
        print(f"\n📂 Output files saved to: {EXPORT_FOLDER}/")
        print("🎯 Next step: Use the dictionary filtering cell to remove common words")

# Run the batch processing
process_all_xlsx_files()

Found 7 xlsx files to process
Target language codes: ['pt-br', 'pt-BR', 'en-us', 'en-gb', 'en-GB', 'es-es', 'es-ES', 'en-US']

📁 Processing file: 2023.03.15_ONE_MORE_GATE_TB.xlsx
🎮 Extracted game name: ONE

  🌐 Trying language code: pt-br (normalized: pt-br)
Processing started at: 2025-09-14 19:21:11

Filter configuration:
  - Skip identical translations: False
  - Skip square brackets: False
  - Skip all caps: False
  - Skip WIP markers: True
  - Tokenization language: portuguese
Detected file type: excel
Excel columns: ['key', 'en-us', 'fr-fr', 'zh-cn', 'de-de', 'es-es']
Sheet used: default
  ⏭️  Language code pt-br not found in file columns - skipping

  🌐 Trying language code: pt-BR (normalized: pt-br)
Processing started at: 2025-09-14 19:21:11

Filter configuration:
  - Skip identical translations: False
  - Skip square brackets: False
  - Skip all caps: False
  - Skip WIP markers: True
  - Tokenization language: portuguese
Detected file type: excel
Excel columns: ['key', 'en-us',

# Merge both token files

Output : single list merged from the TB list + TM list.
Purpose: Useful to avoid problematic non-translations in the TM (élément_FR, élément[WIP]_ES), and add the curated non-translation terms from the terminology base (Wabbit_FR = Wabbit_ES).

In [ ]:
TXT_PATH1 = r"C:\Users\Nelso\Downloads\spanishTB_tokens.txt" #from TB
TXT_PATH2 = r"C:\Users\Nelso\Downloads\spanish_tokens.txt" #from TM
# Merge two text files into one with unique tokens
def merge_token_files(file1: str, file2: str, output_file: str):
    """Merge two token files into one, ensuring unique tokens"""
    if not os.path.exists(file1) or not os.path.exists(file2):
        raise FileNotFoundError("One or both token files do not exist.")
    
    tokens = set()
    
    # Read first file
    with open(file1, 'r', encoding='utf-8') as f:
        for line in f:
            tokens.add(line.strip())
    
    # Read second file
    with open(file2, 'r', encoding='utf-8') as f:
        for line in f:
            tokens.add(line.strip())
    
    # Write unique tokens to output file
    with open(output_file, 'w', encoding='utf-8') as f:
        for token in sorted(tokens):
            f.write(token + '\n')
    
    print(f"Merged {len(tokens)} unique tokens into: {output_file}")

# Merge the two token files
merge_token_files(TXT_PATH1, TXT_PATH2, r"C:\Users\Nelso\Downloads\merged_spanish_tokens.txt")

# Filter words appearing in a common language dictionary

## Filtering v2.0
This new algorithm includes morphological patterns of the AFF files to improve the matching rules and remove more common language words from the Ankama dictionary.
* Hunspell resources : https://hunspell.memoq.com/
* AFF (affix morphological patterns) documentation : https://manpages.ubuntu.com/manpages/focal/man5/hunspell.5.html

In [ ]:
import re
from typing import Set, Dict, List, Tuple
LANG_CODE = "es-es"  # Language code to process

PATH_Ankama_tokens = "output/es-es_TOUCH_tokens_20250914_192342.txt"  # Path to the Ankama tokens file
#PATH_Ankama_tokens = EXPORT_PATH  # Use the previously generated tokens file

DIC_FOLDER = "dics"
dic_lang_paths = {
    # es : os path + dic folder + es + es_ES.dic
    "es": os.path.join(DIC_FOLDER, "es_dic", "es", "es_ES.dic"),
    "fr": os.path.join(DIC_FOLDER, "fr_dic", "fr_FR.dic"),
    "pt": os.path.join(DIC_FOLDER, "pt_dic", "pt_BR", "pt_BR.dic"),
    "en": os.path.join(DIC_FOLDER, "en_dic", "en_GB.dic")
}

# Define Hunspell dic based on LANG_CODE
PATH_Hunspell_dic = dic_lang_paths.get(LANG_CODE[:2])  # Get the first two letters (e.g., 'es' from 'es-es')
if not PATH_Hunspell_dic or not os.path.exists(PATH_Hunspell_dic):
    raise FileNotFoundError(f"Hunspell .dic file for language '{LANG_CODE}' not found in paths: {dic_lang_paths}")

AFF_FILE_PATH = dic_lang_paths.get(LANG_CODE[:2]).replace('.dic', '.aff') if dic_lang_paths.get(LANG_CODE[:2]) else None  # Path to .aff file

# Replace 'tokens' with 'filtered_tokens' and add timestamp in input PATH_Ankama_tokens
if 'tokens' in PATH_Ankama_tokens:
    FILTERED_OUTPUT_PATH = PATH_Ankama_tokens.replace('tokens', f'filtered_tokens')
else:
    FILTERED_OUTPUT_PATH = Path(PATH_Ankama_tokens).stem + '_filtered_tokens.txt'

def parse_aff_file(aff_file_path: str) -> Dict:
    """Parse Hunspell .aff file and extract affix rules"""
    affixes = {'PFX': {}, 'SFX': {}}
    
    with open(aff_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    current_affix = None
    current_type = None
    
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
            
        parts = line.split()
        if not parts:
            continue
            
        # Parse prefix/suffix header definitions (e.g., "PFX a Y 2")
        if parts[0] in ['PFX', 'SFX'] and len(parts) >= 3:
            affix_type = parts[0]
            flag = parts[1]
            cross_product = parts[2] == 'Y'
            
            # Check if this is a header line (has count) or rule line
            if len(parts) >= 4:
                try:
                    # Try to parse as count - if successful, this is a header line
                    count = int(parts[3])
                    # This is a header line
                    if flag not in affixes[affix_type]:
                        affixes[affix_type][flag] = {
                            'cross_product': cross_product,
                            'rules': []
                        }
                    current_affix = flag
                    current_type = affix_type
                    continue
                except ValueError:
                    # Not a number, so this is a rule line
                    pass
            
            # Parse affix rule: PFX/SFX flag strip add condition
            if len(parts) >= 4 and current_affix == flag and current_type == affix_type:
                strip = parts[2] if parts[2] != '0' else ''
                add = parts[3] if parts[3] != '0' else ''
                condition = parts[4] if len(parts) > 4 else '.'
                
                if current_affix in affixes[current_type]:
                    affixes[current_type][current_affix]['rules'].append({
                        'strip': strip,
                        'add': add,
                        'condition': condition
                    })
    
    return affixes

def condition_matches(word: str, condition: str, is_prefix: bool = True) -> bool:
    """Check if word matches the affix condition pattern"""
    if condition == '.':
        return True
    
    try:
        if is_prefix:
            # For prefixes, check the beginning of the word
            return bool(re.match(f'^{condition}', word))
        else:
            # For suffixes, check the end of the word
            return bool(re.search(f'{condition}$', word))
    except re.error:
        # If regex fails, do simple string matching
        if is_prefix:
            return word.startswith(condition.replace('[^', '').replace(']', ''))
        else:
            return word.endswith(condition.replace('[^', '').replace(']', ''))

def generate_word_forms(base_word: str, flags: str, affixes: Dict) -> Set[str]:
    """Generate all possible word forms using affix rules"""
    word_forms = {base_word}  # Always include the base word
    
    if not flags:
        return word_forms
    
    # Process each flag character
    for flag in flags:
        # Apply prefixes
        if flag in affixes['PFX']:
            prefix_rules = affixes['PFX'][flag]['rules']
            for rule in prefix_rules:
                if condition_matches(base_word, rule['condition'], is_prefix=True):
                    # Apply prefix rule
                    if rule['strip']:
                        if base_word.startswith(rule['strip']):
                            modified_word = rule['add'] + base_word[len(rule['strip']):]
                            word_forms.add(modified_word)
                    else:
                        modified_word = rule['add'] + base_word
                        word_forms.add(modified_word)
        
        # Apply suffixes
        if flag in affixes['SFX']:
            suffix_rules = affixes['SFX'][flag]['rules']
            for rule in suffix_rules:
                if condition_matches(base_word, rule['condition'], is_prefix=False):
                    # Apply suffix rule
                    if rule['strip']:
                        if base_word.endswith(rule['strip']):
                            modified_word = base_word[:-len(rule['strip'])] + rule['add']
                            word_forms.add(modified_word)
                    else:
                        modified_word = base_word + rule['add']
                        word_forms.add(modified_word)
    
    return word_forms

def filter_tokens_by_dictionary_with_affixes(txt_file_path: str, dic_file_path: str, aff_file_path: str, output_dic_path: str):
    """
    Enhanced version that uses Hunspell affix rules for better matching
    
    Args:
        txt_file_path: Path to the txt file with tokens (one per line)
        dic_file_path: Path to the dic file (first line is token count, rest are tokens)
        aff_file_path: Path to the .aff file with affix rules
        output_dic_path: Path where the filtered dic file will be saved
    """
    if not os.path.exists(txt_file_path):
        raise FileNotFoundError(f"Token file not found: {txt_file_path}")
    
    if not os.path.exists(dic_file_path):
        raise FileNotFoundError(f"Dictionary file not found: {dic_file_path}")
        
    if not os.path.exists(aff_file_path):
        raise FileNotFoundError(f"Affix file not found: {aff_file_path}")
    
    # Parse affix rules
    print(f"Parsing affix rules from: {aff_file_path}")
    affixes = parse_aff_file(aff_file_path)
    prefix_count = sum(len(rules['rules']) for rules in affixes['PFX'].values())
    suffix_count = sum(len(rules['rules']) for rules in affixes['SFX'].values())
    print(f"Loaded {len(affixes['PFX'])} prefix flags ({prefix_count} rules) and {len(affixes['SFX'])} suffix flags ({suffix_count} rules)")
    
    # Read tokens from txt file - preserve original case
    print(f"Reading tokens from: {txt_file_path}")
    original_txt_tokens = []  # Keep original case
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            token = line.strip()
            if token:
                original_txt_tokens.append(token)  # Preserve original case
    
    print(f"Loaded {len(original_txt_tokens)} tokens from txt file")
    
    # Read dictionary file and generate all word forms
    print(f"Reading dictionary and generating word forms from: {dic_file_path}")
    with open(dic_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    if not lines:
        raise ValueError("Dictionary file is empty")
    
    # First line is the token count
    original_count = lines[0].strip()
    print(f"Dictionary token count: {original_count}")
    
    # Generate all possible word forms from dictionary (in lowercase for matching)
    all_dictionary_forms = set()
    processed_entries = 0
    
    for line in lines[1:]:
        line = line.strip()
        if not line:
            continue
            
        processed_entries += 1
        if processed_entries % 1000 == 0:
            print(f"Processed {processed_entries} dictionary entries...")
        
        # Parse dictionary entry
        if '/' in line:
            base_word, flags = line.split('/', 1)
        else:
            base_word, flags = line, ''
        
        # Generate all word forms for this base word (lowercase for matching)
        word_forms = generate_word_forms(base_word.lower(), flags, affixes)
        all_dictionary_forms.update(word_forms)
    
    print(f"Generated {len(all_dictionary_forms)} unique word forms from {processed_entries} dictionary entries")
    
    # Filter txt tokens - remove those that match any dictionary form
    # Compare lowercase versions but keep original case for output
    filtered_tokens = []
    removed_count = 0
    sample_removals = []
    
    for original_token in original_txt_tokens:  # Use original case tokens
        if original_token.lower() in all_dictionary_forms:  # Compare with lowercase
            removed_count += 1
            if len(sample_removals) < 10:
                sample_removals.append(original_token)  # Show original case in samples
        else:
            filtered_tokens.append(original_token)  # Keep original case
    
    # Show some examples of removed tokens
    if sample_removals:
        print(f"Sample removed tokens: {', '.join(sample_removals[:5])}{'...' if len(sample_removals) > 5 else ''}")
    
    print(f"Removed {removed_count} tokens that match dictionary word forms")
    print(f"Remaining tokens: {len(filtered_tokens)}")
    
    # Write filtered tokens as dictionary file (preserving original case)
    with open(output_dic_path, 'w', encoding='utf-8') as f:
        f.write(str(len(filtered_tokens)) + '\n')
        for token in filtered_tokens:  # These already have original case
            f.write(token + '\n')
    
    print(f"Filtered tokens saved as dictionary to: {output_dic_path}")
    
    return {
        'original_txt_tokens': len(original_txt_tokens),
        'dictionary_base_words': processed_entries,
        'generated_word_forms': len(all_dictionary_forms),
        'removed_tokens': removed_count,
        'remaining_tokens': len(filtered_tokens)
    }

# Test the enhanced function
print("="*70)
print("TESTING ENHANCED DICTIONARY FILTERING WITH AFFIX RULES")
print("="*70)


if os.path.exists(AFF_FILE_PATH):
    try:
        result = filter_tokens_by_dictionary_with_affixes(
            PATH_Ankama_tokens,      # txt file with tokens to filter
            PATH_Hunspell_dic,    # dic file
            AFF_FILE_PATH,           # aff file with rules
            FILTERED_OUTPUT_PATH
        )
        
        print("\nENHANCED FILTERING RESULTS:")
        print("="*50)
        print(f"Original txt tokens: {result['original_txt_tokens']}")
        print(f"Dictionary base words: {result['dictionary_base_words']}")
        print(f"Generated word forms: {result['generated_word_forms']}")
        print(f"Removed tokens: {result['removed_tokens']}")
        print(f"Remaining tokens: {result['remaining_tokens']}")
        
        # Calculate improvement
        improvement = result['generated_word_forms'] - result['dictionary_base_words']
        print(f"Affix expansion factor: {result['generated_word_forms'] / result['dictionary_base_words']:.2f}x")
        print(f"Additional word forms from affixes: {improvement}")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print(f"Affix file not found: {AFF_FILE_PATH}")
    print("Please provide the correct path to the .aff file")

FileNotFoundError: Hunspell .dic file for language 'es-es' not found in paths: {'es': 'dics\\es_dic\\es\\es_ES.dic', 'fr': 'dics\\fr_dic\\fr_FR.dic', 'pt': 'dics\\pt_dic\\pt_BR\\pt_BR.dic', 'en': 'dics\\en_dic\\en_GB.dic'}

# Enhanced Language File Processor - Complete Summary

## Features

The script now includes **comprehensive filtering** with multiple advanced conditions to ensure high-quality token extraction.

### Supported File Types
- **Excel files** (`.xlsx`, `.xls`): Language code as column name
- **XLIFF files** (`.xliff`, `.xlf`, `.xml`): Language code in `source-language` or `target-language` attributes

### Key Functionality
1. **File Type Detection**: Automatically detects file type based on extension
2. **Language Matching**: 
   - Excel: Extracts from column matching the language code
   - XLIFF: Extracts from `<source>` or `<target>` elements based on language attributes

### **COMPREHENSIVE Filtering System**
3. **Square Bracket Filtering**: Ignores entries where source text contains `[.+]` pattern
4. **Target = Source Filtering**: Ignores entries where target text equals source text
5. **All-Caps Target Filtering**: **NEW** - Ignores entries where target text is entirely in uppercase
6. **HTML Tag Removal**: **NEW** - Removes HTML tags and decodes HTML entities before tokenization
7. **Hyperlink & Email Removal**: Removes URLs and email addresses before tokenization
8. **Token Edge Cleaning**: **NEW** - Removes leading/trailing apostrophes and hyphens from tokens
9. **Short Token Filtering**: Removes tokens with length < 3 characters
10. **Same Character Chain Filtering**: Removes tokens that are chains of the same character (e.g., "aaa", "zzZZzz")
11. **Number-Only Token Filtering**: **NEW** - Removes tokens that consist only of digits
12. **Time Pattern Filtering**: **NEW** - Removes tokens matching `\d+(PA|PM|AM|AL)` pattern
13. **Digit-Word Pattern Filtering**: **NEW** - Removes tokens matching `\d+-\w+` pattern (e.g., "123-neutral")
14. **Enhanced Punctuation**: **NEW** - Includes º character in punctuation list
15. **Tokenization**: Splits by whitespace and punctuation, preserving hyphens (`-`) and apostrophes (`'`)
16. **Export**: Saves unique tokens (case-sensitive) to text file, one per line

### Usage
```python
# Basic usage
tokens = process_file(file_path, language_code)

# With custom output path
tokens = process_file(file_path, language_code, output_path)
```

### Example Advanced Filtering Results
**Input Processing:**
- ✅ **"Hola mundo"** → `['Hola', 'mundo']`
- ❌ **"[Debug] test"** → Skipped (square brackets in source)
- ❌ **"Same text"** → Skipped (target equals source)
- ❌ **"TODO EN MAYÚSCULAS"** → Skipped (all caps target)
- ✅ **HTML content** → Tags removed, entities decoded
- ✅ **"'Resistencia 'Robo'"** → `['Resistencia', 'Robo']` (edges cleaned)
- ❌ **Number tokens: "123", "456"** → Filtered out (numbers only)
- ❌ **Time patterns: "3PM", "10AM"** → Filtered out (time pattern)
- ❌ **Digit-word: "123-neutral"** → Filtered out (digit-word pattern)
- ✅ **"25º celsius"** → `['celsius']` (º treated as punctuation)

**Final Result:** Only meaningful, clean tokens ≥ 3 characters from appropriate entries

# Draft tests (unitary tests TODO)

In [ ]:
# Quick verification test for the fixes
print("="*50)
print("VERIFYING FIXES")
print("="*50)

# Test the corrected time pattern function
test_time_tokens = ["3PM", "10AM", "5PA", "12AL"]
print("Testing corrected time patterns:")
for token in test_time_tokens:
    matches = matches_time_pattern(token)
    print(f"'{token}' -> matches time pattern: {matches}")

# Test º character removal
test_text = "Temperature: 25º celsius"
print(f"\nTesting º removal:")
print(f"Original: '{test_text}'")
tokens = tokenize_text(test_text)
print(f"Tokens: {sorted(tokens)}")

# Test all filtering combined
test_combined_text = "Meeting at 3PM, temperature 25º, status: 123-neutral, numbers 456"
print(f"\nTesting combined filtering:")
print(f"Original: '{test_combined_text}'")
tokens_combined = tokenize_text(test_combined_text)
print(f"Final tokens: {sorted(tokens_combined)}")

print("\nExpected results:")
print("- Time patterns (3PM) should be filtered out")
print("- Numbers (456) should be filtered out") 
print("- Digit-word patterns (123-neutral) should be filtered out")
print("- º should be treated as punctuation")
print("- Only meaningful words should remain")

In [ ]:
# Test the fix for HTML br and p tag handling
print("="*60)
print("TESTING HTML BR AND P TAG HANDLING FIX")
print("="*60)

# Test cases that demonstrate the issue and fix
test_html_cases = [
    "Ankama&lt;br&gt;&lt;br&gt;1.",
    "Word1&lt;br&gt;Word2",
    "Start&lt;p&gt;Middle&lt;/p&gt;End",
    "Text&lt;br/&gt;More text",
    "Line1&lt;BR&gt;Line2",  # Test case insensitive
    "Para&lt;P class='test'&gt;Content&lt;/P&gt;After",
    "Normal text without HTML tags"
]

print("Testing HTML tag removal with br/p handling:")
for text in test_html_cases:
    cleaned = remove_html_tags(text)
    tokens = tokenize_text(text)
    print(f"Original: '{text}'")
    print(f"Cleaned:  '{cleaned}'")
    print(f"Tokens:   {sorted(tokens)}")
    print()

# Specific test for the reported issue
print("="*40)
print("SPECIFIC TEST FOR REPORTED ISSUE")
print("="*40)

issue_text = "Ankama&lt;br&gt;&lt;br&gt;1."
print(f"Testing: '{issue_text}'")

# Before fix (simulate): would result in "Ankama1"
# After fix: should result in separate tokens
cleaned_text = remove_html_tags(issue_text)
final_tokens = tokenize_text(issue_text)

print(f"HTML removed: '{cleaned_text}'")
print(f"Final tokens: {sorted(final_tokens)}")
print(f"✅ Issue fixed: 'Ankama' and other meaningful tokens are separate" if 'Ankama' in final_tokens else "❌ Issue not fixed")

# Test with a more complex example
complex_html = "Company&lt;br&gt;&lt;br&gt;Address&lt;p&gt;City&lt;/p&gt;Country123"
print(f"\nComplex example: '{complex_html}'")
complex_tokens = tokenize_text(complex_html)
print(f"Tokens: {sorted(complex_tokens)}")
print("Expected: Company, Address, City, Country123 should be separate tokens")

In [ ]:
# Test the new ignore_identical_translation parameter
print("="*70)
print("TESTING ignore_identical_translation PARAMETER")
print("="*70)

# Create test data with identical translations
test_data_identical = {
    'key': ['greeting', 'same1', 'same2', 'different'],
    'fr-fr': ['Bonjour', 'Same Text', 'Identical', 'Source Text'],
    'es-es': ['Hola', 'Same Text', 'Identical', 'Target Text']  # First two are identical to source
}

df_identical = pd.DataFrame(test_data_identical)
df_identical.to_excel("test_identical.xlsx", index=False)
print("Test Excel file with identical translations created!")
print("Test data:")
print(df_identical.to_string(index=False))

# Test with ignore_identical_translation=True (default)
print(f"\n1. Testing with ignore_identical_translation=True (default):")
try:
    tokens_ignore_true = process_file("test_identical.xlsx", "es-es", "tokens_ignore_true.txt")
    print(f"Tokens with ignore=True: {sorted(tokens_ignore_true)}")
    print("Expected: 'Same Text' and 'Identical' should be skipped")
except Exception as e:
    print(f"Error: {e}")

# Test with ignore_identical_translation=False
print(f"\n2. Testing with ignore_identical_translation=False:")
try:
    tokens_ignore_false = process_file("test_identical.xlsx", "es-es", "tokens_ignore_false.txt", ignore_identical_translation=False)
    print(f"Tokens with ignore=False: {sorted(tokens_ignore_false)}")
    print("Expected: 'Same Text' and 'Identical' should be included")
except Exception as e:
    print(f"Error: {e}")

# Show the difference
if 'tokens_ignore_true' in locals() and 'tokens_ignore_false' in locals():
    additional_tokens = tokens_ignore_false - tokens_ignore_true
    print(f"\nAdditional tokens when ignore_identical_translation=False: {sorted(additional_tokens)}")

# Also test with XLIFF
test_xliff_identical = """<?xml version="1.0" encoding="UTF-8"?>
<xliff version="1.2" xmlns="urn:oasis:names:tc:xliff:document:1.2">
    <file datatype="plaintext" original="test" source-language="fr-fr" target-language="es-es">
        <body>
            <trans-unit id="test.1">
                <source>Hello World</source>
                <target>Hola Mundo</target>
            </trans-unit>
            <trans-unit id="test.2">
                <source>Same Text</source>
                <target>Same Text</target>
            </trans-unit>
            <trans-unit id="test.3">
                <source>Identical</source>
                <target>Identical</target>
            </trans-unit>
        </body>
    </file>
</xliff>"""

with open("test_identical.xliff", "w", encoding="utf-8") as f:
    f.write(test_xliff_identical)

print(f"\n3. Testing XLIFF with ignore_identical_translation=True:")
try:
    xliff_tokens_true = process_file("test_identical.xliff", "es-es", "xliff_tokens_true.txt")
    print(f"XLIFF tokens with ignore=True: {sorted(xliff_tokens_true)}")
except Exception as e:
    print(f"Error: {e}")

print(f"\n4. Testing XLIFF with ignore_identical_translation=False:")
try:
    xliff_tokens_false = process_file("test_identical.xliff", "es-es", "xliff_tokens_false.txt", ignore_identical_translation=False)
    print(f"XLIFF tokens with ignore=False: {sorted(xliff_tokens_false)}")
except Exception as e:
    print(f"Error: {e}")

# Clean up test files
print("\nCleaning up test files...")
test_files = [
    "test_identical.xlsx", "test_identical.xliff",
    "tokens_ignore_true.txt", "tokens_ignore_false.txt",
    "xliff_tokens_true.txt", "xliff_tokens_false.txt"
]
for file in test_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed: {file}")

print("\nParameter test completed!")
print("\nSUMMARY:")
print("- ignore_identical_translation=True (default): Skips entries where target equals source")
print("- ignore_identical_translation=False: Includes all entries, even identical translations")
print("- This allows users to control whether to include identical translations in their token extraction")

In [1]:
import re
from itertools import product

def demorph_string(input_string):
    """
    Expand morphological patterns in localization strings.
    
    Supports two pattern types:
    1. Tilde patterns: {~X...} where X is a letter and ... is suffix
    2. Square bracket patterns: {[N*]?option1:option2} where N is a digit
    
    Args:
        input_string (str): String containing morphological patterns
        
    Returns:
        str: String with all variations joined by spaces
    """
    
    def extract_tilde_patterns(text):
        """Extract all tilde morphological patterns from a word."""
        pattern_regex = r'\{~([^}]+)\}'
        matches = re.findall(pattern_regex, text)
        parsed_patterns = []
        for match in matches:
            # Split by ~ to handle multiple patterns in the same braces
            sub_patterns = match.split('~')
            for sub_pattern in sub_patterns:
                if len(sub_pattern) >= 1:
                    letter = sub_pattern[0]
                    suffix = sub_pattern[1:] if len(sub_pattern) > 1 else ""
                    parsed_patterns.append((letter, suffix))
        return parsed_patterns
    
    def extract_bracket_patterns(text):
        """Extract all bracket patterns from a word."""
        # Pattern: {[digit*]?option1:option2} or {[~digit]?option1:option2}
        pattern_regex = r'\{\[([~]?\d+\*?)\]\?([^:}]*):([^}]*)\}'
        matches = re.findall(pattern_regex, text)
        return matches
    
    def generate_tilde_variations(base_word, patterns):
        """Generate variations for tilde patterns."""
        # Remove patterns from base word to get the root
        root = re.sub(r'\{~[^}]+\}', '', base_word)
        
        # Check if root should be excluded (if 's' or 'm' patterns present)
        pattern_letters = [p[0] for p in patterns]
        exclude_root = 's' in pattern_letters or 'm' in pattern_letters
        
        # If no patterns, return the original word
        if not patterns:
            return [base_word]
        
        variations = []
        
        # Group patterns by type
        gender_patterns = [(letter, suffix) for letter, suffix in patterns if letter in 'mf']
        number_patterns = [(letter, suffix) for letter, suffix in patterns if letter in 'sp']
        
        # Handle gender+number combinations
        if gender_patterns and number_patterns:
            # We need all 4 combinations: masc sing, fem sing, masc plural, fem plural
            
            # 1. Masculine singular (root) - only if not excluded
            if not exclude_root:
                variations.append(root)

            # 2. Masculine singular with masculine suffix
            for g_letter, g_suffix in gender_patterns:
                if g_letter == 'm':
                    male_root = root + g_suffix
                    variations.append(male_root)

            # 3. Feminine singular (root + feminine suffix)
            for g_letter, g_suffix in gender_patterns:
                if g_letter == 'f':
                    variations.append(root + g_suffix)
            
            # 4. Masculine plural (root + plural suffix)  
            for n_letter, n_suffix in number_patterns:
                if n_letter == 'p':
                    variations.append(root + n_suffix)
            
            # 5. Feminine plural (root + feminine suffix + plural suffix)
            for (g_letter, g_suffix), (n_letter, n_suffix) in product(gender_patterns, number_patterns):
                if g_letter == 'f' and n_letter == 'p':
                    variations.append(root + g_suffix + n_suffix)
                    
        else:
            # Handle simple cases (no combinations needed)
            
            # If root should be included, add it first
            if not exclude_root:
                variations.append(root)
            
            # Add individual pattern variations
            for letter, suffix in patterns:
                variation = root + suffix
                variations.append(variation)
        
        # Remove duplicates while preserving order
        seen = set()
        unique_variations = []
        for var in variations:
            if var not in seen:
                seen.add(var)
                unique_variations.append(var)
        
        return unique_variations
    
    def generate_bracket_variations(base_word, bracket_patterns):
        """Generate variations for bracket patterns."""
        if not bracket_patterns:
            return [base_word]
        
        current_variations = [base_word]
        
        for pattern_match, option1, option2 in bracket_patterns:
            new_variations = []
            
            # Build the regex pattern correctly
            pattern_to_replace = r'\{\['  # {[
            pattern_to_replace += re.escape(pattern_match)  # pattern (escaped)
            pattern_to_replace += r'\]\?'  # ]?
            pattern_to_replace += re.escape(option1)  # option1 (escaped)
            pattern_to_replace += ':'  # :
            pattern_to_replace += re.escape(option2)  # option2 (escaped)
            pattern_to_replace += r'\}'  # }
            
            for current_var in current_variations:
                # For the pattern {[N*]?option1:option2}:
                # Generate variation 1: condition true -> use option1 (usually the base/unmarked form)
                var1 = re.sub(pattern_to_replace, option1, current_var, count=1)
                if var1 not in new_variations:
                    new_variations.append(var1)
                
                # Generate variation 2: condition false -> use option2 (usually the marked form)
                var2 = re.sub(pattern_to_replace, option2, current_var, count=1)
                if var2 not in new_variations:
                    new_variations.append(var2)
            
            current_variations = new_variations
        
        return current_variations

    # Find all words with patterns (both types)
    word_pattern_regex = r'\S*\{[~\[][^}]+\}(?:\{[~\[][^}]+\})*'
    
    def replace_word_patterns(match):
        word_with_patterns = match.group(0)
        
        # Check what type of patterns we have
        bracket_patterns = extract_bracket_patterns(word_with_patterns)
        tilde_patterns = extract_tilde_patterns(word_with_patterns)
        
        if bracket_patterns and not tilde_patterns:
            # Only bracket patterns
            variations = generate_bracket_variations(word_with_patterns, bracket_patterns)
        elif tilde_patterns and not bracket_patterns:
            # Only tilde patterns
            variations = generate_tilde_variations(word_with_patterns, tilde_patterns)
        elif bracket_patterns and tilde_patterns:
            # Both types - handle bracket first, then tilde
            bracket_variations = generate_bracket_variations(word_with_patterns, bracket_patterns)
            final_variations = []
            for var in bracket_variations:
                if extract_tilde_patterns(var):
                    tilde_vars = generate_tilde_variations(var, extract_tilde_patterns(var))
                    final_variations.extend(tilde_vars)
                else:
                    final_variations.append(var)
            variations = final_variations
        else:
            # No patterns found (shouldn't happen with our regex)
            variations = [word_with_patterns]
        
        return ' '.join(variations)
    
    # Replace all pattern words with their variations
    result = re.sub(word_pattern_regex, replace_word_patterns, input_string)
    
    return result

In [4]:
# Test all cases from the test suite with clear input/output display
test_cases = [
    # test_basic_suffix_patterns
    ("Apariencia{[~1]?s:} de montura", "Apariencia Apariencias de montura"),
    ("Transmutaci{[~1]?ones:ón}", "Transmutación Transmutaciones"),
    ("Fragmento{[~1]?s:} de Relíquia{[~1]?s:}", "Fragmentos Fragmento de Relíquias Relíquia"),
    
    # test_english_plurals
    ("Display Window{[~1]?s:} & Workshop{[~1]?s:}", "Display Windows Window & Workshops Workshop"),
    
    # test_gender_patterns
    ("Costume d'ouvri{[1*]?ère:er} de l'usine", "Costume d'ouvrier d'ouvrière de l'usine"),
    ("Título: Campeã{[1*]?:o} do Torneio de Verão", "Título: Campeã Campeão do Torneio de Verão"),
    ("Titre : Dragonisat{[1*]?rice:eur} Ultime", "Titre : Dragonisatrice Dragonisateur Ultime"),
    
    # test_other_digits
    ("Title: Ultimate Dragonizer{[3*]?:}", "Title: Ultimate Dragonizer"),
    ("Título: Dragonizador{[2*]?a:} definitivo", "Título: Dragonizadora Dragonizador definitivo"),
    
    # test_standalone_pattern
    ("Título: {[1*]?Dragonizadora Suprema:Dragonizador Supremo}", "Título: Dragonizadora Suprema Dragonizador Supremo"),
    
    # test_tilde_patterns (key cases with grammar codes)
    ("Misi{~són~pones}", "Misión Misiones"),
    
    # test_additional_cases
    ("%1 posede %2 personaje{~ps} en este servidor", "%1 posede %2 personaje personajes en este servidor"),
    ("Possedé{~fe}{~ps}", "Possedé Possedée Possedés Possedées"),
    ("%1 misi{~són}{~pones} pendiente{~ps}", "%1 misión misiones pendiente pendientes"),
    ("Espos{~mo}{~fa}", "Esposo Esposa"),
    
    # test_any_digit_patterns
    ("Jugador{[3*]?a:} premium", "Jugador Jugadora premium"),
    ("Vendedor{[42*]?a:} oficial", "Vendedora Vendedor oficial"),
    ("Administrador{[999*]?a:} del sistema", "Administradora Administrador del sistema"),
]

# Test the demorph function with all test cases
# Modified to check if result and expected have same set of words regardless of order
print("Testing demorph function with ALL test cases:")
print("=" * 60)

def words_match(result, expected):
    """Check if two strings have the same set of unique words regardless of order."""
    result_words = set(result.split())
    expected_words = set(expected.split())
    return result_words == expected_words

passed = 0
total = 0

for input_str, expected in test_cases:
    result = demorph_string(input_str)
    
    # Check both exact match and word set match
    exact_match = result == expected
    words_same = words_match(result, expected)
    
    total += 1
    if words_same:
        passed += 1
    
    print(f"Input:    {input_str}")
    print(f"Expected: {expected}")
    print(f"Result:   {result}")
    
    # Show different types of matches
    if exact_match:
        print(f"Match:    Exact ✅")
    elif words_same:
        print(f"Match:    Same words (different order) ✅")
    else:
        print(f"Match:    Failed ❌")
        # Show word difference for debugging
        expected_words = set(expected.split())
        result_words = set(result.split())
        if expected_words != result_words:
            missing = expected_words - result_words
            extra = result_words - expected_words
            if missing:
                print(f"          Missing words: {missing}")
            if extra:
                print(f"          Extra words: {extra}")
    
    print("-" * 40)

print(f"\nSummary: {passed}/{total} tests passed ({passed/total*100:.1f}%)")

Testing demorph function with ALL test cases:
Input:    Apariencia{[~1]?s:} de montura
Expected: Apariencia Apariencias de montura
Result:   Apariencias Apariencia de montura
Match:    Same words (different order) ✅
----------------------------------------
Input:    Transmutaci{[~1]?ones:ón}
Expected: Transmutación Transmutaciones
Result:   Transmutaciones Transmutación
Match:    Same words (different order) ✅
----------------------------------------
Input:    Fragmento{[~1]?s:} de Relíquia{[~1]?s:}
Expected: Fragmentos Fragmento de Relíquias Relíquia
Result:   Fragmentos Fragmento de Relíquias Relíquia
Match:    Exact ✅
----------------------------------------
Input:    Display Window{[~1]?s:} & Workshop{[~1]?s:}
Expected: Display Windows Window & Workshops Workshop
Result:   Display Windows Window & Workshops Workshop
Match:    Exact ✅
----------------------------------------
Input:    Costume d'ouvri{[1*]?ère:er} de l'usine
Expected: Costume d'ouvrier d'ouvrière de l'usine
Result:  